# This notebook uses light-gbm to tackle the Parkinson's Freezing of Gait (FOG) Prediction problem

Thanks to JEROENVDD* for pointing out the utility of seglearn and tsflex.

tsflex - "tsflex is a toolkit for flexible time series processing & feature extraction, that is efficient and makes few assumptions about sequence data." (https://github.com/predict-idlab/tsflex)

seglearn - "provides an integrated pipeline for segmentation, feature extraction, feature processing, and final estimator. Seglearn provides a flexible approach to multivariate time series and related contextual (meta) data for classification, regression, and forecasting problems" (https://github.com/dmbee/seglearn)

*https://www.kaggle.com/code/jeroenvdd/time-series-tsflex

In [1]:
# Install tsflex and seglearn - pull in all the whl files that are required for this notebook
!pip install tsflex -q --no-index --find-links=file:///kaggle/input/time-series-tools
!pip install seglearn -q --no-index --find-links=file:///kaggle/input/time-series-tools

# Load Datasets

Note the two labelled datasets that we have access to, from the competition webpage:
"The tDCS FOG (tdcsfog) dataset, comprising data series collected in the lab, as subjects completed a FOG-provoking protocol."
"The DeFOG (defog) dataset, comprising data series collected in the subject's home, as subjects completed a FOG-provoking protocol"

In [2]:
# import commonly used libraries
import os
import numpy as np
import pandas as pd
import pathlib
from tqdm.auto import tqdm
from sklearn import *
import glob
import matplotlib.pyplot as plt
import pickle
import sklearn

# using light-gbm as our model
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import average_precision_score

# import time-series libraries
import seglearn
from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper
from sklearn.model_selection import GroupKFold # add k fold


In [3]:
# link to dataset
dataset_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

# create train and test directories
train = glob.glob(dataset_path + 'train/**/**')
test = glob.glob(dataset_path + 'test/**/**')

# create additional csvs
subjects = pd.read_csv(dataset_path + 'subjects.csv')
tasks = pd.read_csv(dataset_path + 'tasks.csv')
SampleSubmission = pd.read_csv(dataset_path + 'sample_submission.csv')

# combine the two datasets mentioned - tdcsfog and defog
tdcsfog_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv')
defog_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv')

tdcsfog_metadata['Module']='tdcsfog'
defog_metadata['Module']='defog'

metadata=pd.concat([tdcsfog_metadata,defog_metadata])

In [4]:
# feature engineering

# For Tasks: create feature to capture duration of event
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[-1] for c in tasks.columns]
tasks = tasks.reset_index()

# fill data with KMeans
tasks['t_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(tasks[tasks.columns[1:]])

# For Subjects: fill data
subjects = subjects.fillna(0).groupby('Subject').median()
subjects = subjects.reset_index()

# fill data with KMeans
subjects['s_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(subjects[subjects.columns[1:]])
subjects=subjects.rename(columns={'Visit':'s_Visit','Age':'s_Age','YearsSinceDx':'s_YearsSinceDx','UPDRSIII_On':'s_UPDRSIII_On','UPDRSIII_Off':'s_UPDRSIII_Off','NFOGQ':'s_NFOGQ'})

complex_featlist = ['Medication', 'Test', 'Visit', 's_Age', 's_NFOGQ', 's_UPDRSIII_Off', 's_UPDRSIII_On', 's_Visit', 's_YearsSinceDx', 's_kmeans']

metadata_complex=metadata.merge(subjects,how='left',on='Subject').copy()
metadata_complex['Medication']=metadata_complex['Medication'].factorize()[0]

In [5]:
# using tsflex library for time-series feature extraction
# MultipleFeatureDescriptors class from seglearn for defining multiple feature descriptors

# define a window of 5k
windowSize = [5_000]

# create feature descriptors for base features
baseFeatures = MultipleFeatureDescriptors(
    functions = seglearn_feature_dict_wrapper(seglearn.feature_functions.base_features()),
    series_names = ['AccV', 'AccML', 'AccAP'],
    windows = windowSize,
    strides = windowSize, # even stride
)

# create feature descriptors for EMG features
emgFeatures = seglearn.feature_functions.emg_features()
del emgFeatures['simple square integral'] # delete duplicate category

emgFeatures = MultipleFeatureDescriptors(
    functions = seglearn_feature_dict_wrapper(emgFeatures),
    series_names = ['AccV', 'AccML', 'AccAP'],
    windows = windowSize,
    strides = windowSize, # even stride
)

fc = FeatureCollection([baseFeatures, emgFeatures])

In [6]:
def readFunction(f):
    try:
        # import the csv file as a pandas dataframe
        df = pd.read_csv(f, index_col="Time", usecols=['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn' , 'Walking'])
        
        # edit features
        df['Id'] = f.split('/')[-1].split('.')[0]
        df['Module'] = pathlib.Path(f).parts[-2]
        df['Time_frac'] = (df.index/df.index.max()).values #currently the index of data is actually "Time"
        
        # merge vecs
        df = pd.merge(df, tasks[['Id','t_kmeans']], how='left', on='Id').fillna(-1)
        df = pd.merge(df, metadata_complex[['Id','Subject']+['Visit','Test','Medication','s_kmeans']], how='left', on='Id').fillna(-1)
        df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
        df = df.merge(df_feats, how="left", left_index=True, right_index=True)
        df.fillna(method="ffill", inplace=True)
        return df
    
    except: pass

# if not os.path.exists(dataset_path + 'data.pkl'):
#     # run read function
train = pd.concat([readFunction(f) for f in tqdm(train)]).fillna(0)
train = train.reset_index(drop=True)
# the file is too big to be saved in a pickle file!
#     # Save dataframe to a pickle file
#     try:
#         with open('/kaggle/working/data.pkl', 'wb') as file:
#             pickle.dump(train, file)
#         print('saved file to: ' + '/kaggle/working/data.pkl')
#     else:
#         print("couldn't save, likely read-only")
# else:
#     # Load dataframe from the pickle file
#     with open('/kaggle/working/data.pkl', 'rb') as file:
#         train = pickle.load(file)
#     print('loaded file from: ' + '/kaggle/working/data.pkl')

cols = [c for c in train.columns if c not in ['Id','Subject','Module', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]
pcols = ['StartHesitation', 'Turn' , 'Walking']
scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']

  0%|          | 0/970 [00:00<?, ?it/s]

## Model Training

In [ ]:
# customize for light-gbm 
class LGBMMultiOutputRegressor(MultiOutputRegressor):
    
    def fit(self, X, y, eval_set=None, **fit_params):
        
        # copy base estimator for each output 
        self.estimators_ = [sklearn.base.clone(self.estimator) for indx in range(0,y.shape[1])]
        
        # Fit the estimators
        for i, estimator in enumerate(self.estimators_):
            
            if eval_set is not None:
                eval_set_i = (eval_set[0], eval_set[1][:, i])
                fit_params['eval_set'] = [eval_set_i]
            
            estimator.fit(X, y[:, i], **fit_params)

        return self

# patching ap scoring for light-gbm
def lgbm_ap(y_true, y_pred):
    
    score = average_precision_score(y_true, y_pred)
    
    return 'average_precision', score, True

In [ ]:
# setting up for 5-fold cross validation
numFolds = 5
kFolds = GroupKFold(numFolds)
groups = kFolds.split(train, groups = train.Subject)
regs, cvs = [], []

for fold, (trainIndex,testIndex) in enumerate(tqdm(groups, total = numFolds)):
    trainIndex = pd.Series(trainIndex).sample(n = 200,random_state = 1).values
    
    # Create a light-gbm regressor
    base_regressor = lgb.LGBMRegressor(
    max_depth = 8,
    learning_rate = 0.2,
    n_estimators = 300,
    subsample = 0.99,
    min_child_weight = 3.12,
    colsample_bytree = 0.5
    )

    # Wrap the base regressor with the MultiOutputRegressor
    multioutput_regressor = LGBMMultiOutputRegressor(base_regressor)

    # set up data
    # train
    xTrain = train.loc[trainIndex,cols].to_numpy()
    yTrain = train.loc[trainIndex,pcols].to_numpy()
    
    # test
    xTest = train.loc[testIndex,cols].to_numpy()
    yTest = train.loc[testIndex,pcols].to_numpy()
    
    # define early stopping callback
    early_stopping = lgb.callback.early_stopping(
        stopping_rounds = 25, # number of rounds to wait before stopping if no improvement
        verbose = False, # print message when stopping
        first_metric_only = True,
    )
    
    # train light-gbm
    multioutput_regressor.fit(
    xTrain,
    yTrain,
    eval_set = (xTest,yTest),
    eval_metric=lgbm_ap,
    early_stopping_rounds=25
    )
    regs.append(multioutput_regressor)
    cv=metrics.average_precision_score(yTest, multioutput_regressor.predict(xTest).clip(0.0,1.0))
    cvs.append(cv)
print(cvs)

## Predict for test

In [ ]:
SampleSubmission['t'] = 0
SampleSubmission = []

for f in test:
    df = pd.read_csv(f)
    df.set_index('Time', drop = True, inplace = True)

    df['Id'] = f.split('/')[-1].split('.')[0]
    df['Time_frac'] = (df.index/df.index.max()).values#currently the index of data is actually "Time"
    df = pd.merge(df, tasks[['Id','t_kmeans']], how = 'left', on = 'Id').fillna(-1)
    df = pd.merge(df, metadata_complex[['Id','Subject'] + ['Visit','Test','Medication','s_kmeans']], how = 'left', on = 'Id').fillna(-1)
    df_feats = fc.calculate(df, return_df = True, include_final_window = True, approve_sparsity = True, window_idx = "begin")
    df = df.merge(df_feats, how = "left", left_index = True, right_index = True)
    df.fillna(method = "ffill", inplace = True)
    
    res_vals=[]

    for i_fold in range(numFolds):
        res_val=np.round(regs[i_fold].predict(df[cols]).clip(0.0,1.0),3)
        res_vals.append(np.expand_dims(res_val,axis = 2))
    res_vals=np.mean(np.concatenate(res_vals,axis = 2),axis = 2)
    res = pd.DataFrame(res_vals, columns = pcols)
    
    df = pd.concat([df,res], axis=1)
    df['Id'] = df['Id'].astype(str) + '_' + df.index.astype(str)
    SampleSubmission.append(df[scols])
    
SampleSubmission = pd.concat(SampleSubmission)
SampleSubmission = pd.merge(SampleSubmission[['Id']], SampleSubmission, how='left', on='Id').fillna(0.0)
SampleSubmission[scols].to_csv('submission.csv', index=False)

In [ ]:
SampleSubmission.head()